In [1]:
from pathlib import Path
import xarray as xr
import numpy as np

In [2]:
basepath = Path("/workspace/data/africa_v0.3/train/")

minicube_paths = list(basepath.glob("*/*.nc"))

minicube = xr.open_dataset(minicube_paths[3])

In [13]:
s2_avail = minicube.s2_avail
print(type(s2_avail), s2_avail.shape)
index_avail = np.where(s2_avail == 1)[0]
print(index_avail)
print(index_avail[1:] - index_avail[:-1] - 5)
A = index_avail[1:] - index_avail[:-1] - 5
C = index_avail + A
print(np.where(A > 0)[0])

<class 'xarray.core.dataarray.DataArray'> (450,)
[  4   9  14  19  24  29  34  39  44  49  54  59  64  69  74  79  84  89
  94  99 104 109 114 119 124 129 134 139 144 149 154 159 164 169 174 179
 184 189 194 199 204 209 214 219 224 229 234 239 244 249 254 259 264 269
 274 279 284 289 294 299 304 309 314 319 324 329 334 339 344 349 354 359
 364 369 374 379 384 389 394 399 404 409 414 419 424 429 434 439 444 449]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


ValueError: operands could not be broadcast together with shapes (90,) (89,) 

In [6]:
s2_avail = minicube.s2_avail
time = np.array([i for i in range(4, len(s2_avail), 5)])


# Select the days with available data
time = np.array([i for i in range(4, len(s2_avail), 5)])
print(s2_avail)
s2_avail = s2_avail.isel(time=time)
print(s2_avail)

<xarray.DataArray 's2_avail' (time: 450)>
[450 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2019-01-16 2019-01-17 ... 2020-04-09
Attributes:
    coordinates:  sentinel:product_id
<xarray.DataArray 's2_avail' (time: 90)>
[90 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2019-01-20 2019-01-25 ... 2020-04-09
Attributes:
    coordinates:  sentinel:product_id


In [21]:
        # Detect the missing day
        # Index of the availables days
        index_avail = np.where(s2_avail == 1)[0]
        missing = index_avail[1:] - index_avail[:-1] - 5
        # index of the missing days (due to cloud or other)
        index_missing = np.where(missing > 0)[0]
        if index_avail[0] % 5 != 4:
            raise Exception("bad assumption on the data, first value s2 =" + str(index_avail[0]))

4
